<img src="https://img-blog.csdn.net/2018092916042238?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2thaWVuMTIyNg==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70" style="width:1000px;height:500px;float:left">

<font color=#8552a1 size= 6>
  tf.Session 
</font>

<hr class="docutils" />

<blockquote><p><font size= 4>
Defined in tensorflow/python/client/session.py.
<p>
A class for running TensorFlow operations.
<p>
A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated. For example:
    </font>
</blockquote>

<table style="width:100%;border-collapse:collapse;background-color:white;">
    <tr style="background-color:rgba(30, 200,100, 0.8)" >
      <th> <font size= 5> Example  </font></th>
    </tr>
<table>

In [4]:
import tensorflow as tf
# Build a graph.
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# Launch the graph in a session.
sess = tf.Session()

# Evaluate the tensor `c`.
print(sess.run(c))


30.0


<blockquote><p><font size= 4>
A session may own resources, such as tf.Variable, tf.QueueBase, and tf.ReaderBase. It is important to release these resources when they are no longer required. To do this, either invoke the tf.Session.close method on the session, or use the session as a context manager. The following two examples are equivalent:
<p>
    <code>
    # Using the `close()` method.
    sess = tf.Session()
    sess.run(...)
    sess.close()

    # Using the context manager.
    with tf.Session() as sess:
    sess.run(...)
  </code>
<p>
The ConfigProto protocol buffer exposes various configuration options for a session. For example, to create a session that uses soft constraints for device placement, and log the resulting placement decisions, create a session as follows:
    <code>
    # Launch the graph in a session that allows soft device placement and
    # logs the placement decisions.
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                            log_device_placement=True))
    </code>
    
   </font>
</blockquote>

<font color=#8552a1 size= 4>
  Supported devices
</font>

<hr class="docutils" />
<blockquote><p><font size= 4>
On a typical system, there are multiple computing devices. In TensorFlow, the supported device types are CPU and GPU. They are represented as strings. For example:
<p>
    "/cpu:0": The CPU of your machine.
<p>
    "/device:GPU:0": The GPU of your machine, if you have one.
<p>
    "/device:GPU:1": The second GPU of your machine, etc.
<p>
If a TensorFlow operation has both CPU and GPU implementations, the GPU devices will be given priority when the operation is assigned to a device. For example, matmul has both CPU and GPU kernels. On a system with devices cpu:0 and gpu:0, gpu:0 will be selected to run matmul.
    </font>
</blockquote>

<font color=#8552a1 size= 4>
  Logging Device placement
</font>

<hr class="docutils" />
<blockquote><p><font size= 4>
To find out which devices your operations and tensors are assigned to, create the session with log_device_placement configuration option set to True.
    </font>
</blockquote>

<table style="width:100%;border-collapse:collapse;background-color:white;">
    <tr style="background-color:rgba(30, 200,100, 0.8)" >
      <th> <font size= 5> Example  </font></th>
    </tr>
<table>

In [5]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[ 22.  28.]
 [ 49.  64.]]


<font color=#8552a1 size= 4>
  Manual device placement
</font>

<hr class="docutils" />
<blockquote><p><font size= 4>
If you would like a particular operation to run on a device of your choice instead of what's automatically selected for you, you can use with tf.device to create a device context such that all the operations within that context will have the same device assignment.
    </font>
</blockquote>

<table style="width:100%;border-collapse:collapse;background-color:white;">
    <tr style="background-color:rgba(30, 200,100, 0.8)" >
      <th> <font size= 5> Example  </font></th>
    </tr>
<table>

In [6]:
# Creates a graph.
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[ 22.  28.]
 [ 49.  64.]]


<blockquote><p><font size= 4>
You will see that now a and b are assigned to cpu:0. Since a device was not explicitly specified for the MatMul operation, the TensorFlow runtime will choose one based on the operation and available devices (gpu:0 in this example) and automatically copy tensors between devices if required.
    </font>
</blockquote>


<font color=#8552a1 size= 4>
  Allowing GPU memory growth
</font>

<hr class="docutils" />
<blockquote><p><font size= 4>
If you would like a particular operation to run on a device of your choice instead of what's automatically selected for you, you can use with tf.device to create a device context such that all the operations within that context will have the same device assignment.
<p>
By default, TensorFlow maps nearly all of the GPU memory of all GPUs (subject to CUDA_VISIBLE_DEVICES) visible to the process. This is done to more efficiently use the relatively precious GPU memory resources on the devices by reducing memory fragmentation.
<p>
In some cases it is desirable for the process to only allocate a subset of the available memory, or to only grow the memory usage as is needed by the process. TensorFlow provides two Config options on the Session to control this.
<p>
The first is the allow_growth option, which attempts to allocate only as much GPU memory based on runtime allocations: it starts out allocating very little memory, and as Sessions get run and more GPU memory is needed, we extend the GPU memory region needed by the TensorFlow process. Note that we do not release memory, since that can lead to even worse memory fragmentation. To turn this option on, set the option in the ConfigProto by:
<code>
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config, ...)
</code>
<p>
The second method is the per_process_gpu_memory_fraction option, which determines the fraction of the overall amount of memory that each visible GPU should be allocated. For example, you can tell TensorFlow to only allocate 40% of the total memory of each GPU by:
<code>
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config, ...)
    </code>
    <p>
This is useful if you want to truly bound the amount of GPU memory available to the TensorFlow process.
</font>
</blockquote>
